- intubation_extubation_before_paired 데이터의 intubationtime 과 extubationtime 사이에 해당하는 변수들을 추가 
- intubation_extubation_before_paired : intubation_extubation_before_reconstruction 파일에서 intubation, extubation 결측값이 없도록 페어링한 파일
- 추가하고 마지막에 컬럼순서 변경 (vital, ventset, gcs, lab_abga 순서대로 추가)

In [5]:
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta
import time
import src.ventilator_mode_mapping.ventilator_mapping as vm

In [6]:
subjectlist1_intextevents = pd.read_csv('./data/intubation_extubation_before_paired.csv', \
                                    parse_dates=['intubationtime', 'extubationtime', \
                                                 'admittime', 'dischtime', 'deathtime','reintubation_eventtime']) 
subjectlist1_intextevents.columns

Index(['Unnamed: 0', 'subject_id', 'stay_id', 'gender', 'los', 'admittime',
       'dischtime', 'deathtime', 'hadm_id', 'intubationtime', 'int_flag_dup',
       'int_time_diff', 'flag_intudup2_filttime', 'extubationtime',
       'extubationcause', 'ext_flag_dup', 'ext_time_diff',
       'flag_extudup2_filttime', 'reint_marker', 'mvtime',
       'reintubation_eventtime', 'reintubationtime', 'seq_num', 'mvtime_state',
       'final_event', 'ext_to_death', 'ext_to_disch', 'disch_to_death',
       'class_code', 'class', 'duration'],
      dtype='object')

####  필요한 컬럼만 가져오기 
- 사용할 컬럼 목록

- subject_id: 환자 고유번호 (출처: hosp_patients)
- hadm_id: 병원 입원 고유번호 (출처: hosp_admissions)
- stay_id: 중환자실 입원 고유번호. 발관 시간 기준으로 생성. 발관시간 NULL일 경우 삽관 이벤트 활용 (출처: icu_icustays)
- gender: 성별
- los: Length of Stay (days)
- admittime: 입원시각 (출처: hosp_admissions)
- intubationtime: 삽관시각 (출처: icu_procedureevents)
- extubationtime: 발관시각 (출처: icu_procedureevents) 
- extubationcause: 발관 사유 (Planned Extubation, Unplanned Extubation (patient-initiated), Unplanned Extubation (patient-uninitiated)) (출처: icu_procedureevents)
- dischtime: 퇴원시각 (출처: hosp_admissions)
- deathtime: 사망시각 (출처: hosp_admissions)
- seq_num: 삽관-발관 이벤트의 시퀀스 순서(1~n)
- mvtime: 삽관시각과 발관시각의 시간차 (단위: 분)
- reintubationtime: 발관 후 재삽관까지 걸린 시간 (단위: 분)
- mvtime_state (boolean): intext_duration이 1440분 이내 (<= 24시간)이면 mechanical ventilation (True)으로 분류 
- class_code: 케이스별로 고유한 13개 코드로 데이터를 분류 (하단 테이블 참조)
- class: class_code 기반으로 (Extubation) failure, non-failure, death의 3개 라벨로 분류함

In [7]:
# 필요한 컬럼만 가져오기 
# 사용할 컬럼 목록
columns_to_use = ['subject_id', 'stay_id', 'gender', 'los', 'admittime', 'dischtime',
       'deathtime', 'hadm_id', 'intubationtime',
        'extubationtime', 'extubationcause',
        'reint_marker', 'mvtime', 'reintubation_eventtime',
       'reintubationtime', 'seq_num', 'mvtime_state', 'final_event',
       'ext_to_death', 'ext_to_disch', 'disch_to_death', 'class_code',
       'class']

# 필요한 컬럼들만 선택
subjectlist1_intextevents = subjectlist1_intextevents[columns_to_use]
subjectlist1_intextevents

,subject_id,stay_id,gender,los,admittime,dischtime,deathtime,hadm_id,intubationtime,extubationtime,...,reintubation_eventtime,reintubationtime,seq_num,mvtime_state,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,NaT,NaN,1,False,True,10775.00,10775.00,0.0,11,nonfailure
1,10002013,39060235.0,F,1.314352,2160-05-18 07:45:00,2160-05-23 13:30:00,NaT,23581541,2160-05-18 10:00:53,2160-05-18 18:00:00,...,NaT,NaN,1,True,True,NaN,6930.00,NaN,11,nonfailure
2,10002428,35479615.0,F,10.977222,2156-05-11 14:49:00,2156-05-22 14:16:00,NaT,23473524,2156-05-11 14:49:34,2156-05-20 10:45:00,...,NaT,NaN,1,False,True,NaN,3091.00,NaN,11,nonfailure
3,10002428,38875437.0,F,7.032894,2156-04-12 14:16:00,2156-04-29 16:26:00,NaT,28662225,2156-04-19 18:11:19,2156-04-22 17:10:00,...,NaT,NaN,1,False,True,NaN,10036.00,NaN,11,nonfailure
4,10002760,31831386.0,M,1.045868,2141-04-20 07:15:00,2141-04-24 13:31:00,NaT,28094813,2141-04-20 13:20:46,2141-04-21 08:00:00,...,NaT,NaN,1,True,True,NaN,4651.00,NaN,11,nonfailure
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24773,19999068,30143796.0,M,6.765324,2161-08-24 04:10:00,2161-09-02 19:00:00,NaT,21606769,2161-08-25 15:34:00,2161-08-28 13:35:00,...,NaT,NaN,1,False,True,NaN,7525.00,NaN,11,nonfailure
24774,19999287,35165301.0,F,4.705752,2197-08-03 20:58:00,2197-08-18 15:37:00,NaT,20175828,2197-08-04 00:02:00,2197-08-07 09:20:00,...,NaT,NaN,1,False,True,NaN,16217.00,NaN,11,nonfailure
24775,19999442,32336619.0,M,6.950370,2148-11-19 10:00:00,2148-12-04 16:25:00,NaT,26785317,2148-11-19 19:00:00,2148-11-20 14:15:00,...,NaT,NaN,1,True,True,NaN,20290.00,NaN,11,nonfailure
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,NaT,NaN,1,False,True,-173.25,-173.25,0.0,1221,death


### 변수 추가 
-> 변수의 chartttime이 intubationtime부터 extubationtime에 포함된 데이터 
- 추가 순서 : abga, ventset, vital, gcs (순서는 중요하지 않음)


In [8]:
sub_vitals = pd.read_csv('./origin_data/sub_vitals.csv')
sub_gcs_me = pd.read_csv('./origin_data/sub_gcs_notna.csv')
sub_ventset = pd.read_csv('./origin_data/sub_ventset.csv')
sub_abga_unique = pd.read_csv('./origin_data/sub_abga_unique.csv')

C:\Users\inhae\AppData\Local\Temp\ipykernel_20424\2803373384.py:4: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  sub_abga_unique = pd.read_csv('./origin_data/sub_abga_unique.csv')


#### abga 변수추가 
추가한 변수:  pCO2, pO2, SaO2, HCO3-, pH, Base excess

In [9]:
subjectlist1_intextevents['intubationtime'] = pd.to_datetime(subjectlist1_intextevents['intubationtime'])
subjectlist1_intextevents['extubationtime'] = pd.to_datetime(subjectlist1_intextevents['extubationtime'])

sub_abga_unique['charttime'] = pd.to_datetime(sub_abga_unique['charttime'])
# itemid와 데이터 컬럼명을 매핑
itemid_to_column = {
    50802: 'Base excess',
    50817: 'SaO2',
    50818: 'pCO2',
    50820: 'pH',
    50821: 'pO2',
    50882: 'HCO3-'
}

# Step 2: 필터링 및 데이터 가져오기
results = []

for idx, row in subjectlist1_intextevents.iterrows():
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    hadm_id = row['hadm_id']
    
    # 48시간 조건 적용
    time_diff = extubationtime - intubationtime
    if time_diff > timedelta(hours=48):
        intubationtime = extubationtime - timedelta(hours=48)
    
    # 해당 기간 동안의 데이터를 필터링
    mask = (sub_abga_unique['hadm_id'] == hadm_id) & \
           (sub_abga_unique['charttime'] >= intubationtime) & \
           (sub_abga_unique['charttime'] <= extubationtime)
    
    filtered_data = sub_abga_unique[mask]

    # Step 3: 각 항목에 맞는 데이터를 가져와서 결과를 정리
    for _, lab_row in filtered_data.iterrows():
        itemid = lab_row['itemid']
        column_name = itemid_to_column.get(itemid, None)
        if column_name:
            result_row = {
                'subject_id': row['subject_id'],
                'hadm_id': row['hadm_id'],
                'stay_id': row['stay_id'],
                'los': row['los'],
                'intubationtime': row['intubationtime'],
                'extubationtime': row['extubationtime'],
                'extubationcause': row['extubationcause'],
                'admittime': row['admittime'],
                'dischtime': row['dischtime'],
                'deathtime': row['deathtime'],
                'mvtime': row['mvtime'],
                'gender': row['gender'],
                'reintubationtime' :row['reintubationtime'],
                'seq_num': row['seq_num'],
                'mvtime_state': row['mvtime_state'],
                'class_code': row['class_code'],
                'class': row['class'],
                'charttime': lab_row['charttime'],
                column_name: lab_row['valuenum'], 
            }
            results.append(result_row)

# Step 4: 결과 테이블 생성
abga_result_df = pd.DataFrame(results)

abga_result_df = abga_result_df.groupby(['hadm_id', 'charttime']).agg({
    'subject_id': 'first', #
    'stay_id': 'first',#
    'los': 'first',
    'gender': 'first',
    'dischtime':'first',
    'deathtime': 'first',
    'intubationtime': 'first',
    'extubationtime': 'first',
    'extubationcause':'first',
    'seq_num': 'first',
    'mvtime_state': 'first',
    'class_code': 'first',
    'class': 'first',
    'reintubationtime': 'first',
    'pCO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'pO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'SaO2': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'HCO3-': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'pH': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None,
    'Base excess': lambda x: x.dropna().iloc[0] if not x.dropna().empty else None
}).reset_index()


# 필요한 컬럼들만 출력
final_columns = ['subject_id', 'hadm_id','los', 'stay_id',  'gender', 'admittime','dischtime', 'deathtime','intubationtime', 'extubationtime' ,'extubationcause','seq_num','mvtime','class_code','class'
,'reintubationtime','charttime','pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess']
abga_result_df = abga_result_df.reindex(columns=final_columns, fill_value=float('nan'))
abga_result_df

,subject_id,hadm_id,los,stay_id,gender,admittime,dischtime,deathtime,intubationtime,extubationtime,...,class_code,class,reintubationtime,charttime,pCO2,pO2,SaO2,HCO3-,pH,Base excess
0,14577567,20001361,6.046644,33475095.0,M,NaN,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,2143-05-08 16:21:00,...,11,nonfailure,NaN,2143-05-08 04:41:00,NaN,NaN,NaN,26.0,NaN,NaN
1,14577567,20001361,6.046644,33475095.0,M,NaN,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,2143-05-08 16:21:00,...,11,nonfailure,NaN,2143-05-08 05:22:00,39.0,145.0,NaN,NaN,7.46,4.0
2,19657904,20004357,3.320648,30729609.0,F,NaN,2157-08-12 18:00:00,NaT,2157-08-05 19:00:00,2157-08-07 13:28:00,...,11,nonfailure,NaN,2157-08-06 01:14:00,49.0,99.0,NaN,NaN,7.37,1.0
3,19657904,20004357,3.320648,30729609.0,F,NaN,2157-08-12 18:00:00,NaT,2157-08-05 19:00:00,2157-08-07 13:28:00,...,11,nonfailure,NaN,2157-08-06 04:35:00,NaN,NaN,NaN,29.0,NaN,NaN
4,19657904,20004357,3.320648,30729609.0,F,NaN,2157-08-12 18:00:00,NaT,2157-08-05 19:00:00,2157-08-07 13:28:00,...,11,nonfailure,NaN,2157-08-06 13:55:00,45.0,104.0,NaN,NaN,7.40,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61012,13997063,29995505,19.032245,37541993.0,M,NaN,2123-03-29 15:00:00,NaT,2123-03-06 23:42:00,2123-03-24 21:09:26,...,11,nonfailure,NaN,2123-03-23 12:17:00,35.0,117.0,98.0,NaN,7.45,1.0
61013,13997063,29995505,19.032245,37541993.0,M,NaN,2123-03-29 15:00:00,NaT,2123-03-06 23:42:00,2123-03-24 21:09:26,...,11,nonfailure,NaN,2123-03-24 04:30:00,NaN,NaN,NaN,25.0,NaN,NaN
61014,17660536,29997844,3.110880,34886849.0,M,NaN,2125-03-03 16:21:00,NaT,2125-02-26 16:25:00,2125-03-01 17:09:29,...,121,nonfailure,NaN,2125-02-28 02:17:00,NaN,NaN,NaN,23.0,NaN,NaN
61015,17660536,29997844,3.110880,34886849.0,M,NaN,2125-03-03 16:21:00,NaT,2125-02-26 16:25:00,2125-03-01 17:09:29,...,121,nonfailure,NaN,2125-03-01 05:40:00,NaN,NaN,NaN,27.0,NaN,NaN


#### ventset 변수 추가 
추가한 변수 : respiratory_rate_set, respiratory_rate_total,
                respiratory_rate_spontaneous, minute_volume, tidal_volume_set,
                tidal_volume_observed, tidal_volume_spontaneous, plateau_pressure,
                peep, fio2, flow_rate, ventilator_mode, 
                ventilator_mode_hamilton', ventilator_type, ventilator_scoring

In [10]:
"""
ventilator_mode, ventilator_mode_hamilton 컬럼의 mode를 숫자로 매핑해주는 함수
- CMV -> 4
- SIMV -> 3
- CPAP, PSV -> 2
- SPONT -> 1
- 특정 조합에 해당하는 행 -> 5
"""

# ventilator_mode 매핑 딕셔너리 정의
ventilator_mode_mapping = {
    ('SPONT','Standby'): 1, # SPONT
    ('CPAP/PSV', 'PSV/SBT','CPAP/PSV+ApnVol','CPAP','APRV','CPAP/PPS','CPAP/PSV+ApnPres','APRV/Biphasic+ApnVol','CPAP/PSV+Apn TCPL','Apnea Ventilation'): 2,  # CPAP, PSV
    ('MMV/PSV/AutoFlow','MMV/PSV','SIMV/PSV/AutoFlow','SIMV/PSV','MMV','MMV/AutoFlow','SIMV/PRES','SIMV/AutoFlow','SIMV/VOL','SIMV','PRVC/SIMV'): 3,  # SIMV
    ('CMV/ASSIST/AutoFlow','CMV/ASSIST','PCV+Assist','CMV/AutoFlow','PCV+/PSV','CMV','PCV+','VOL/AC','APV (cmv)','P-CMV','(S) CMV','PRVC/AC','PRES/AC'): 4  # CMV
}

def map_ventilator_mode(value):
    for keys, mapped_value in ventilator_mode_mapping.items():
        if value in keys:
            return mapped_value
    return None


# ventilator_mode_hamilton 매핑 딕셔너리 정의
ventilator_mode_hamilton_mapping = {
    ('SPONT',): 1, # SPONT
    ('APRV','nCPAP-PS','NIV','NIV-ST','VS'): 2,  # CPAP, PSV
    ('ASV','APV (simv)','SIMV'): 3,  # SIMV
    ('APV (cmv)','P-CMV','(S) CMV'): 4  # CMV
}

def map_ventilator_mode_hamilton(value):
    for keys, mapped_value in ventilator_mode_hamilton_mapping.items():
        if value in keys:
            return mapped_value
    return None

def update_ventilator_scoring(df):
    '''
    ventilator_mode, ventilator_mode_hamilton 매핑
    '''

    # ventilator_mode와 ventilator_mode_hamilton이 모두 na인 경우는 None
    df['ventilator_scoring'] = None  # 기본값을 None으로 설정

    # ventilator_mode이 na가 아닌 경우 해당 값을 사용하여 매핑
    mask_mode = df['ventilator_mode'].notna()
    df.loc[mask_mode, 'ventilator_scoring'] = df.loc[mask_mode, 'ventilator_mode'].apply(map_ventilator_mode)

    # ventilator_mode_hamilton가 na가 아닌 경우 해당 값을 사용하여 매핑 (ventilator_mode의 매핑을 덮어쓸 수 있음)
    mask_hamilton = df['ventilator_mode_hamilton'].notna()
    df.loc[mask_hamilton, 'ventilator_scoring'] = df.loc[mask_hamilton, 'ventilator_mode_hamilton'].apply(map_ventilator_mode_hamilton)

    # 특정 조합에 해당하는 행을 5로 맵핑
    pairs = [
        ('PSV/SBT', 'APV (cmv)'),
        ('Standby', 'APV (cmv)'),
        ('CMV/ASSIST', 'SPONT'),
        ('PSV/SBT', 'SPONT'),
        ('CPAP/PSV', 'Ambient'),
        ('CMV/ASSIST/AutoFlow', 'Ambient')
    ]

    for mode, hamilton in pairs:
        df.loc[(df['ventilator_mode'] == mode) & (df['ventilator_mode_hamilton'] == hamilton), 'ventilator_scoring'] = 5

    return df

In [11]:
ventset_patient = vm.update_ventilator_scoring(sub_ventset)
ventset_patient

,Unnamed: 0,subject_id,hadm_id,int_stayid,stay_id,charttime,respiratory_rate_set,respiratory_rate_total,respiratory_rate_spontaneous,minute_volume,...,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,0,10094805,21370539,34488364,34488364,2182-04-25 12:00:00,12.0,24.0,NaN,10.70,...,633.0,NaN,19.0,5.0,50.0,NaN,CMV/ASSIST,NaN,Drager,4.0
1,1,10094805,21370539,34488364,34488364,2182-04-25 16:51:00,12.0,18.0,NaN,11.70,...,626.0,NaN,19.0,5.0,50.0,NaN,CMV/ASSIST,NaN,Drager,4.0
2,2,10094805,21370539,34488364,34488364,2182-04-25 19:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,19.0,NaN,NaN,NaN,NaN,NaN,NaN,None
3,3,10094805,21370539,34488364,34488364,2182-04-25 19:34:00,12.0,15.0,NaN,9.90,...,608.0,NaN,NaN,5.0,50.0,NaN,CMV/ASSIST,NaN,Drager,4.0
4,4,10094805,21370539,34488364,34488364,2182-04-25 23:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,18.0,NaN,NaN,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
825539,825539,10090454,29525590,38669202,38669202,2169-02-10 12:00:00,NaN,27.0,27.0,10.80,...,318.0,320.0,NaN,5.0,40.0,NaN,CPAP/PSV,NaN,Drager,2.0
825540,825540,10090454,29525590,38669202,38669202,2169-02-10 14:00:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,None
825541,825541,10090454,29525590,38669202,38669202,2169-02-10 18:00:00,14.0,26.0,0.0,8.00,...,340.0,NaN,NaN,9.0,100.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
825542,825542,10090454,29525590,38669202,38669202,2169-02-10 18:14:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,None


In [12]:
ventset_patient['charttime'] = pd.to_datetime(ventset_patient['charttime'])
# Step 1: Group ventset_patient by 'hadm_id' once before the loop
ventset_patient_grouped = ventset_patient.groupby('hadm_id')

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each row in subjectlist1_intextevents
for index, row in subjectlist1_intextevents.iterrows():
    hadm_id = row['hadm_id']
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    
    # Determine start_time based on time difference
    time_diff = extubationtime - intubationtime
    start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

    # Check if hadm_id exists in ventset_patient_grouped
    if hadm_id in ventset_patient_grouped.groups:
        # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
        group = ventset_patient_grouped.get_group(hadm_id)
        
        # Step 4: Filter the group by time window only (much smaller subset than the whole dataset)
        filtered_group = group[
            (group['charttime'] >= start_time) &
            (group['charttime'] <= extubationtime)
        ]
        
        # Step 5: Append filtered rows to result
        for _, vent_row in filtered_group.iterrows():
            new_row = row.copy()
            for col in [
                'charttime', 'respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring'
            ]:
                new_row[col] = vent_row.get(col, None)  # Using get to avoid KeyErrors
            optimized_result_list.append(new_row)

# Step 6: Create the final result DataFrame
ventset_result_df = pd.DataFrame(optimized_result_list)
ventset_result_df

,subject_id,stay_id,gender,los,admittime,dischtime,deathtime,hadm_id,intubationtime,extubationtime,...,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,408.0,NaN,22.0,5.1,50.0,35.6,NaN,APV (cmv),Hamilton,4.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,430.0,NaN,23.0,5.2,50.0,32.3,NaN,APV (cmv),Hamilton,4.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaN,NaN
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,331.0,331.0,NaN,5.0,NaN,NaN,NaN,SPONT,NaN,1.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,351.0,351.0,NaN,5.0,50.0,36.0,NaN,SPONT,Hamilton,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,508.0,NaN,27.0,10.0,60.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,NaN,NaN,NaN,12.0,40.0,NaN,NaN,NaN,Drager,NaN
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,503.0,NaN,28.0,13.0,40.0,NaN,CMV/ASSIST/AutoFlow,NaN,Drager,4.0
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,500.0,NaN,27.0,12.0,40.0,NaN,CMV/ASSIST,NaN,Drager,4.0


#### 병합 (abga를 변수로 추가한 테이블과 ventset를 변수로 추가한 테이블과 병합)

In [13]:
# Concatenate the two dataframes
combined_df = pd.concat([abga_result_df, ventset_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
abga_ventset = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess','respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring']
for col in columns_to_add:
    if col not in abga_ventset.columns:
        abga_ventset[col] = float('nan')
abga_ventset

C:\Users\inhae\AppData\Local\Temp\ipykernel_20424\1802575858.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat([abga_result_df, ventset_result_df], ignore_index=True)


,hadm_id,charttime,subject_id,los,stay_id,gender,admittime,dischtime,deathtime,intubationtime,...,tidal_volume_observed,tidal_volume_spontaneous,plateau_pressure,peep,fio2,flow_rate,ventilator_mode,ventilator_mode_hamilton,ventilator_type,ventilator_scoring
0,20001361,2143-05-07 22:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,666.0,650.0,NaN,5.0,NaN,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0
1,20001361,2143-05-08 01:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,701.0,NaN,NaN,5.0,50.0,NaN,CMV/ASSIST/AutoFlow,None,None,4.0
2,20001361,2143-05-08 01:11:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,681.0,NaN,NaN,5.0,50.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0
3,20001361,2143-05-08 04:41:00,14577567,6.046644,33475095.0,M,NaT,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
4,20001361,2143-05-08 05:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,714.0,NaN,NaN,5.0,50.0,NaN,CMV/ASSIST/AutoFlow,None,None,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164861,29999098,2128-06-20 11:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,528.0,NaN,17.0,5.5,60.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0
164862,29999098,2128-06-20 15:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,557.0,NaN,17.0,5.5,60.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0
164863,29999098,2128-06-20 17:26:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
164864,29999098,2128-06-20 19:30:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,495.0,NaN,17.0,5.0,60.0,NaN,CMV/ASSIST/AutoFlow,None,Drager,4.0


#### vital_sign 추가
추가한 변수 : charttime, heart_rate, sbp, dbp, mbp, 
                    resp_rate, temperature, temperature_site, spo2, glucose

In [14]:
sub_vitals['charttime'] = pd.to_datetime(sub_vitals['charttime'])
# Step 1: Split 'sub_vitals' by 'hadm_id' into chunks
hadm_ids = sub_vitals['hadm_id'].unique()

# 원하는 청크 크기 설정 (예: 한 번에 100개 'hadm_id'씩 처리)
chunk_size = 1000

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each chunk
for i in range(0, len(hadm_ids), chunk_size):
    # 청크별로 'hadm_id' 나누기
    chunk_hadm_ids = hadm_ids[i:i + chunk_size]
    
    # Step 4: 해당 청크에 해당하는 'sub_vitals' 데이터만 가져오기
    sub_vitals_chunk = sub_vitals[sub_vitals['hadm_id'].isin(chunk_hadm_ids)]
    
    # Step 5: Group the chunk by 'hadm_id'
    sub_vitals_grouped = sub_vitals_chunk.groupby('hadm_id')

    # Step 6: Process each row in 'subjectlist1_intextevents'
    for index, row in subjectlist1_intextevents.iterrows():
        hadm_id = row['hadm_id']
        intubationtime = row['intubationtime']
        extubationtime = row['extubationtime']
        
        # Determine start_time based on time difference
        time_diff = extubationtime - intubationtime
        start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

        # Check if hadm_id exists in the current chunk
        if hadm_id in sub_vitals_grouped.groups:
            # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
            group = sub_vitals_grouped.get_group(hadm_id)
            
            # Step 7: Filter the group by time window only (much smaller subset than the whole dataset)
            filtered_group = group[
                (group['charttime'] >= start_time) &
                (group['charttime'] <= extubationtime)
            ]
            
            # Step 8: Append filtered rows to result
            for _, vital_row in filtered_group.iterrows():
                new_row = row.copy()
                for col in [
                    'charttime', 'heart_rate', 'sbp', 'dbp', 'mbp', 
                    'resp_rate', 'temperature', 'temperature_site', 'spo2', 'glucose'
                ]:
                    new_row[col] = vital_row.get(col, None)  # Using get to avoid KeyErrors
                optimized_result_list.append(new_row)

# Step 9: Create the final result DataFrame
vitals_result_df = pd.DataFrame(optimized_result_list)

# Display the final result
vitals_result_df

,subject_id,stay_id,gender,los,admittime,dischtime,deathtime,hadm_id,intubationtime,extubationtime,...,charttime,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose
195,10094805,34488364.0,M,5.905463,2182-04-23 19:14:00,2182-05-09 13:20:00,2182-05-09 13:20:00,21370539,2182-04-25 12:22:00,2182-04-28 13:15:00,...,2182-04-26 14:00:00,75.0,99.0,50.0,62.0,25.0,NaN,NaN,97.0,NaN
195,10094805,34488364.0,M,5.905463,2182-04-23 19:14:00,2182-05-09 13:20:00,2182-05-09 13:20:00,21370539,2182-04-25 12:22:00,2182-04-28 13:15:00,...,2182-04-26 15:00:00,80.0,103.0,53.0,65.0,27.0,37.44,Rectal,98.0,NaN
195,10094805,34488364.0,M,5.905463,2182-04-23 19:14:00,2182-05-09 13:20:00,2182-05-09 13:20:00,21370539,2182-04-25 12:22:00,2182-04-28 13:15:00,...,2182-04-26 16:00:00,75.0,94.0,55.0,54.0,26.0,NaN,NaN,98.0,NaN
195,10094805,34488364.0,M,5.905463,2182-04-23 19:14:00,2182-05-09 13:20:00,2182-05-09 13:20:00,21370539,2182-04-25 12:22:00,2182-04-28 13:15:00,...,2182-04-26 17:00:00,88.0,111.0,60.0,73.0,27.0,37.00,Oral,98.0,NaN
195,10094805,34488364.0,M,5.905463,2182-04-23 19:14:00,2182-05-09 13:20:00,2182-05-09 13:20:00,21370539,2182-04-25 12:22:00,2182-04-28 13:15:00,...,2182-04-26 18:00:00,90.0,115.0,66.0,77.0,32.0,37.89,Rectal,99.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24773,19999068,30143796.0,M,6.765324,2161-08-24 04:10:00,2161-09-02 19:00:00,NaT,21606769,2161-08-25 15:34:00,2161-08-28 13:35:00,...,2161-08-28 09:00:00,63.0,100.0,60.0,69.0,11.0,NaN,NaN,99.0,NaN
24773,19999068,30143796.0,M,6.765324,2161-08-24 04:10:00,2161-09-02 19:00:00,NaT,21606769,2161-08-25 15:34:00,2161-08-28 13:35:00,...,2161-08-28 10:00:00,79.0,131.0,77.0,89.0,14.0,NaN,NaN,100.0,NaN
24773,19999068,30143796.0,M,6.765324,2161-08-24 04:10:00,2161-09-02 19:00:00,NaT,21606769,2161-08-25 15:34:00,2161-08-28 13:35:00,...,2161-08-28 11:00:00,79.0,126.0,82.0,92.0,14.0,NaN,NaN,100.0,NaN
24773,19999068,30143796.0,M,6.765324,2161-08-24 04:10:00,2161-09-02 19:00:00,NaT,21606769,2161-08-25 15:34:00,2161-08-28 13:35:00,...,2161-08-28 12:00:00,59.0,117.0,74.0,84.0,12.0,37.17,Oral,100.0,NaN


In [15]:
# vitals_result_df.to_csv('./data/vitals_result_df.csv') 

#### abga_ventset 데이터 프레임과 vital를 변수로 추가한 데이터 프레임 병합

In [16]:
# Concatenate the two dataframes
combined_df = pd.concat([abga_ventset, vitals_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
abga_ventset_vital = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess',
                  
                'respiratory_rate_set', 'respiratory_rate_total', 'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure','peep', 'fio2', 'flow_rate', 
                  'ventilator_mode', 'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
                  
                'heart_rate','sbp'	,'dbp','mbp','resp_rate',	'temperature',	'temperature_site',	'spo2',	'glucose']
for col in columns_to_add:
    if col not in abga_ventset_vital.columns:
        abga_ventset_vital[col] = float('nan')
abga_ventset_vital

,hadm_id,charttime,subject_id,los,stay_id,gender,admittime,dischtime,deathtime,intubationtime,...,ventilator_scoring,heart_rate,sbp,dbp,mbp,resp_rate,temperature,temperature_site,spo2,glucose
0,20001361,2143-05-07 22:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,4.0,77.0,139.0,72.0,93.0,16.0,NaN,None,99.0,NaN
1,20001361,2143-05-07 23:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,NaN,76.0,123.0,82.0,83.0,15.0,NaN,None,98.0,NaN
2,20001361,2143-05-07 23:06:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,NaN,NaN,143.0,66.0,83.0,NaN,NaN,None,NaN,NaN
3,20001361,2143-05-08 00:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,NaN,76.0,120.0,94.0,109.0,15.0,37.50,Axillary,98.0,NaN
4,20001361,2143-05-08 01:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,4.0,80.0,136.0,65.0,85.0,14.0,NaN,None,99.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549099,29999098,2128-06-20 23:30:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN
549100,29999098,2128-06-21 00:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,69.0,120.0,59.0,79.0,16.0,36.44,Oral,97.0,NaN
549101,29999098,2128-06-21 01:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,68.0,117.0,59.0,78.0,16.0,NaN,None,96.0,135.0
549102,29999098,2128-06-21 02:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,69.0,126.0,62.0,84.0,16.0,NaN,None,94.0,193.0


#### gcs 변수추가
추가한 변수 : charttime, gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable

In [17]:
#gcs추가

sub_gcs_me['charttime'] = pd.to_datetime(sub_gcs_me['charttime'])

# Step 1: Group ventset_patient by 'hadm_id' once before the loop
sub1_gcs_grouped = sub_gcs_me.groupby('hadm_id')

# Step 2: Initialize an empty result list
optimized_result_list = []

# Step 3: Process each row in subjectlist1_intextevents
for index, row in subjectlist1_intextevents.iterrows():
    hadm_id = row['hadm_id']
    intubationtime = row['intubationtime']
    extubationtime = row['extubationtime']
    
    # Determine start_time based on time difference
    time_diff = extubationtime - intubationtime
    start_time = extubationtime - timedelta(hours=48) if time_diff > timedelta(hours=48) else intubationtime

    # Check if hadm_id exists in ventset_patient_grouped
    if hadm_id in sub1_gcs_grouped.groups:
        # Retrieve relevant group for the hadm_id (faster than filtering the entire dataset)
        group = sub1_gcs_grouped .get_group(hadm_id)
        
        # Step 4: Filter the group by time window only (much smaller subset than the whole dataset)
        filtered_group = group[
            (group['charttime'] >= start_time) &
            (group['charttime'] <= extubationtime)
        ]
        
        # Step 5: Append filtered rows to result
        for _, gcs_row in filtered_group.iterrows():
            new_row = row.copy()
            for col in [
                'charttime', 'gcs',	'gcs_motor','gcs_verbal','gcs_eyes','gcs_unable'
            ]:
                new_row[col] = gcs_row.get(col, None)  # Using get to avoid KeyErrors
            optimized_result_list.append(new_row)

# Step 6: Create the final result DataFrame
gcs_result_df = pd.DataFrame(optimized_result_list)
gcs_result_df

,subject_id,stay_id,gender,los,admittime,dischtime,deathtime,hadm_id,intubationtime,extubationtime,...,ext_to_disch,disch_to_death,class_code,class,charttime,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,10775.00,0.0,11,nonfailure,2131-01-11 08:00:00,15.0,1.0,0.0,1.0,1.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,10775.00,0.0,11,nonfailure,2131-01-11 10:00:00,15.0,1.0,0.0,2.0,1.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,10775.00,0.0,11,nonfailure,2131-01-11 12:00:00,15.0,5.0,0.0,3.0,1.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,10775.00,0.0,11,nonfailure,2131-01-11 14:00:00,15.0,5.0,0.0,3.0,1.0
0,10001884,37510196.0,F,9.171817,2131-01-07 20:39:00,2131-01-20 05:15:00,2131-01-20 05:15:00,26184834,2131-01-11 04:30:00,2131-01-12 17:40:00,...,10775.00,0.0,11,nonfailure,2131-01-11 16:00:00,15.0,5.0,0.0,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,-173.25,0.0,1221,death,2164-09-16 20:21:00,15.0,1.0,0.0,1.0,1.0
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,-173.25,0.0,1221,death,2164-09-17 00:00:00,15.0,1.0,0.0,1.0,1.0
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,-173.25,0.0,1221,death,2164-09-17 04:00:00,15.0,1.0,0.0,1.0,1.0
24776,19999840,38978960.0,M,5.297766,2164-09-10 13:47:00,2164-09-17 13:42:00,2164-09-17 13:42:00,21033226,2164-09-12 09:40:00,2164-09-17 16:35:15,...,-173.25,0.0,1221,death,2164-09-17 08:08:00,15.0,1.0,0.0,1.0,1.0


### abga_ventset_vital 데이터 프레임과 gcs데이터 프레임 병합

In [18]:
# 병합시도
combined_df = pd.concat([abga_ventset_vital, gcs_result_df], ignore_index=True)

# Group by 'hadm_id' and 'charttime' and combine the columns
vital_ventset_gcs_abga = combined_df.groupby(['hadm_id', 'charttime'], as_index=False).first()

# Add missing columns from both dataframes (in case one is missing a column from the other)
columns_to_add = ['pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess','respiratory_rate_set', 'respiratory_rate_total',
                'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set',
                'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure',
                'peep', 'fio2', 'flow_rate', 'ventilator_mode', 
                'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
                'heart_rate','sbp'	,'dbp','mbp','resp_rate',	'temperature',	'temperature_site',	'spo2',	'glucose','gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable']
for col in columns_to_add:
    if col not in vital_ventset_gcs_abga.columns:
        vital_ventset_gcs_abga[col] = float('nan')
vital_ventset_gcs_abga

,hadm_id,charttime,subject_id,los,stay_id,gender,admittime,dischtime,deathtime,intubationtime,...,resp_rate,temperature,temperature_site,spo2,glucose,gcs,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable
0,20001361,2143-05-07 22:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,16.0,NaN,None,99.0,NaN,NaN,NaN,NaN,NaN,NaN
1,20001361,2143-05-07 23:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,15.0,NaN,None,98.0,NaN,NaN,NaN,NaN,NaN,NaN
2,20001361,2143-05-07 23:06:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20001361,2143-05-08 00:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,15.0,37.50,Axillary,98.0,NaN,15.0,4.0,0.0,2.0,1.0
4,20001361,2143-05-08 01:00:00,14577567,6.046644,33475095.0,M,2143-05-04 14:55:00,2143-05-18 16:58:00,NaT,2143-05-07 21:07:00,...,14.0,NaN,None,99.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572617,29999098,2128-06-20 23:30:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572618,29999098,2128-06-21 00:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,16.0,36.44,Oral,97.0,NaN,NaN,NaN,NaN,NaN,NaN
572619,29999098,2128-06-21 01:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,16.0,NaN,None,96.0,135.0,NaN,NaN,NaN,NaN,NaN
572620,29999098,2128-06-21 02:00:00,15129243,11.071678,33475199.0,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-20 11:25:00,2128-06-10 09:30:00,...,16.0,NaN,None,94.0,193.0,NaN,NaN,NaN,NaN,NaN


### 컬럼 순서 변경 -> 기존컬럼 + vital + ventset + gcs + lab_abga 순서대로

In [19]:
# 컬럼 순서 변경
column_order = [
    'subject_id', 'hadm_id', 'stay_id', 'los', 'gender','admittime',  'dischtime', 
    'intubationtime', 'extubationtime', 'extubationcause', 'mvtime', 'seq_num', 
     'mvtime_state', 'class_code', 'class', 'charttime',
    'heart_rate','sbp','dbp','mbp', 'resp_rate', 'temperature', 'temperature_site', 'spo2', 'glucose', # vital
     'respiratory_rate_set', 'respiratory_rate_total', 'respiratory_rate_spontaneous', 'minute_volume', 'tidal_volume_set', # ventset
    'tidal_volume_observed', 'tidal_volume_spontaneous', 'plateau_pressure', 'peep', 'fio2', 'flow_rate', 
    'ventilator_mode', 'ventilator_mode_hamilton', 'ventilator_type', 'ventilator_scoring',
    'gcs','gcs_motor','gcs_verbal','gcs_eyes','gcs_unable', # gcs
    'pCO2', 'pO2', 'SaO2', 'HCO3-', 'pH', 'Base excess' # abga
    
    
]

# Checking which columns exist in the DataFrame and reordering them
existing_columns = [col for col in column_order if col in vital_ventset_gcs_abga.columns]
vital_ventset_gcs_abga= vital_ventset_gcs_abga[existing_columns]
vital_ventset_gcs_abga

,subject_id,hadm_id,stay_id,los,gender,admittime,dischtime,intubationtime,extubationtime,extubationcause,...,gcs_motor,gcs_verbal,gcs_eyes,gcs_unable,pCO2,pO2,SaO2,HCO3-,pH,Base excess
0,14577567,20001361,33475095.0,6.046644,M,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14577567,20001361,33475095.0,6.046644,M,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14577567,20001361,33475095.0,6.046644,M,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14577567,20001361,33475095.0,6.046644,M,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,...,4.0,0.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,14577567,20001361,33475095.0,6.046644,M,2143-05-04 14:55:00,2143-05-18 16:58:00,2143-05-07 21:07:00,2143-05-08 16:21:00,Planned Extubation,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572617,15129243,29999098,33475199.0,11.071678,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-21 03:01:13,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572618,15129243,29999098,33475199.0,11.071678,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-21 03:01:13,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572619,15129243,29999098,33475199.0,11.071678,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-21 03:01:13,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
572620,15129243,29999098,33475199.0,11.071678,M,2128-06-09 23:45:00,2128-06-20 11:25:00,2128-06-10 09:30:00,2128-06-21 03:01:13,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
vital_ventset_gcs_abga.info() #(578287, 51)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 572622 entries, 0 to 572621
Data columns (total 51 columns):
 #   Column                        Non-Null Count   Dtype         
---  ------                        --------------   -----         
 0   subject_id                    572622 non-null  int64         
 1   hadm_id                       572622 non-null  int64         
 2   stay_id                       572622 non-null  float64       
 3   los                           572622 non-null  float64       
 4   gender                        572622 non-null  object        
 5   admittime                     544696 non-null  datetime64[ns]
 6   dischtime                     572622 non-null  datetime64[ns]
 7   intubationtime                572622 non-null  datetime64[ns]
 8   extubationtime                572622 non-null  datetime64[ns]
 9   extubationcause               412569 non-null  object        
 10  mvtime                        544696 non-null  float64       
 11  seq_num      

In [21]:
# 최종 테이블 저장
vital_ventset_gcs_abga.to_csv('./data/before_vital_ventset_gcs_abga.csv') 